## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic-03-LCEL")

LangSmith 추적을 시작합니다.
[프로젝트명]
project_name='CH01-Basic-03-LCEL'
LANGSMITH_PROJECT: CH01-Basic-03-LCEL


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [8]:
# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(
#     model="gpt-4.1-nano",
#     temperature=0.1,
# )
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import SecretStr
import os


# 키 로드
api_key = os.getenv("GEMINI_API_KEY")

# LLM 생성
# 모델 종류: https://ai.google.dev/gemini-api/docs/models?hl=ko
# gemini-2.5-pro
# gemini-2.5-flash
# gemini-2.5-flash-lite
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=SecretStr(str(api_key)),
    temperature=0.5,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [9]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=SecretStr(str(api_key)),
    temperature=0.5,
)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [10]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [11]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='물론입니다! 인공지능(AI) 모델의 학습 원리를 아주 쉽게, 비유를 통해 설명해 드릴게요.\n\n**인공지능을 \'아주 똑똑하지만, 처음에는 아무것도 모르는 학생\'**이라고 생각해보세요. 이 학생을 가르치는 과정이 바로 AI 모델의 \'학습\'입니다.\n\n### 학습에 필요한 4가지 핵심 요소\n\n이 학생을 가르치려면 4가지가 필요합니다.\n\n1.  **학생 (AI 모델):** 지식을 담을 수 있는 텅 빈 뇌. 처음에는 아무것도 모르는 백지상태입니다.\n2.  **교과서와 문제집 (학습 데이터):** 학생이 공부할 자료입니다. 예를 들어 \'고양이\'와 \'개\'를 구분하는 학생을 가르친다면, 수백만 장의 고양이 사진과 개 사진이 바로 교과서가 됩니다. 각 사진에는 \'이건 고양이\', \'이건 개\'라는 정답(레이블)이 달려 있습니다.\n3.  **풀어야 할 문제 (목표):** 학생의 목표는 \'사진을 보고 고양이인지 개인지 맞히는 것\'입니다.\n4.  **선생님의 가르침과 오답노트 (학습 알고리즘):** 학생이 문제를 틀렸을 때, 왜 틀렸는지 알려주고 정답에 가까워지도록 방향을 알려주는 규칙과 과정입니다.\n\n---\n\n### AI의 학습 과정 (5단계)\n\n이제 이 학생이 어떻게 공부하는지 단계별로 살펴보겠습니다.\n\n#### **1단계: 일단 추측하기 (찍기)**\n학생(AI)에게 고양이 사진 한 장을 보여줍니다. 학생은 아직 배운 게 없으니 그냥 찍습니다.\n> **학생(AI):** "음... 이건 **개**인 것 같아요!"\n\n#### **2단계: 정답과 비교하기**\n우리는 학생에게 정답이 적힌 교과서(데이터)를 보여줍니다.\n> **우리:** "땡! 정답은 **고양이**야."\n\n#### **3단계: 얼마나 틀렸는지 확인하기 (오차 계산)**\n학생은 자신이 얼마나 터무니없는 답을 했는지 깨닫습니다. \'개\'라고 확신했는데 \'고양이\'였으니, \'엄청나게 많이 틀렸다\'고 생각합니다.\n\

아래는 스트리밍을 출력하는 예시 입니다.

In [12]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

네, 인공지능 모델의 학습 원리를 **'시험공부 하는 학생'**에 비유해서 아주 쉽게 설명해 드릴게요.

---

### **인공지능은 어떻게 똑똑해질까요? (feat. 열공하는 학생)**

여기 '인공지능(AI)'이라는 학생이 한 명 있다고 상상해 보세요. 이 학생의 목표는 '강아지와 고양이 사진을 완벽하게 구분하는 시험'에 합격하는 것입니다.

#### **1단계: 방대한 양의 문제집 풀기 (데이터 학습)**

*   **학생:** 시험을 잘 보려면 먼저 문제집을 풀어봐야겠죠? 선생님은 학생에게 **수백만 장의 '강아지 사진'과 '고양이 사진'이 담긴 문제집(학습 데이터)**을 줍니다. 각 사진에는 '이건 강아지야', '이건 고양이야'라고 **정답(레이블)**이 적혀 있습니다.
*   **AI:** 이 문제집을 처음부터 끝까지 훑어보며 강아지는 보통 어떤 특징(귀 모양, 코, 수염 등)을 가졌고, 고양이는 어떤 특징을 가졌는지 어렴풋이 감을 잡기 시작합니다.

#### **2단계: 모의고사 보기 (예측)**

*   **학생:** 이제 어느 정도 공부했으니, 정답이 가려진 사진으로 **모의고사(예측)**를 봅니다. 학생은 사진을 보고 "음... 이건 70% 확률로 강아지 같아!"라고 답을 제출합니다.
*   **AI:** 학습한 데이터를 바탕으로 새로운 사진을 보고 나름의 판단을 내립니다.

#### **3단계: 채점하고 오답노트 작성하기 (손실 함수)**

*   **학생:** 선생님이 학생의 답을 **채점**합니다. 정답은 '고양이'였는데 '강아지'라고 답했으니 "땡! 틀렸어!"라고 알려줍니다. 심지어 "정답과 아주 멀리 떨어진, 완전히 틀린 답을 냈구나"라며 **얼마나 심하게 틀렸는지(오차/손실)** 점수로 알려줍니다.
*   **AI:** 이 채점 과정을 **'손실 함수(Loss Function)'**라고 부릅니다. AI가 내놓은 예측이 실제 정답과 얼마나 다른지를 숫자로 계산하는 과정이죠. 오차(손실)가 클수록 더 많이 틀렸다는 뜻입니다.

###

### 출력파서(Output Parser)


In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [14]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [15]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'네, 인공지능 모델의 학습 원리를 아주 쉽게, **\'눈치 빠른 수험생\'**에 비유해서 설명해 드릴게요.\n\n### 인공지능은 \'눈치 빠른 수험생\'과 같아요\n\n여기 엄청나게 많은 문제와 정답이 담긴 문제집을 풀어야 하는 수험생(인공지능 모델)이 있다고 상상해보세요. 이 수험생의 목표는 처음 보는 수능 시험(새로운 데이터)도 잘 푸는 것입니다.\n\n학습 과정은 다음과 같은 4단계로 이루어집니다.\n\n---\n\n#### 1단계: 일단 풀어보기 (추측, Prediction)\n\n수험생은 아직 아무것도 모르기 때문에, 일단 문제를 보고 감으로 답을 찍습니다.\n\n*   **수험생:** 고양이 사진을 보고 "음... 이건 개인가?" 하고 추측합니다. (아직 귀 모양, 수염 같은 특징을 전혀 모르는 상태)\n*   **인공지능:** 모델은 수많은 숫자(가중치, 파라미터)로 이루어진 텅 빈 상태입니다. 고양이 사진 데이터를 입력받고, 이 숫자들을 이용해 일단 아무 결과나 내놓습니다. (예: "개일 확률 60%, 고양이일 확률 40%")\n\n#### 2단계: 채점하기 (오차 계산, Loss)\n\n문제를 푼 뒤에는 정답지와 비교해서 채점을 합니다. 얼마나 틀렸는지 확인하는 과정이죠.\n\n*   **수험생:** 정답지를 보니 \'고양이\'라고 되어 있습니다. "아, \'개\'라고 생각했는데 틀렸네. 많이 틀렸군." 하고 깨닫습니다.\n*   **인공지능:** 모델이 내놓은 답("개 60%, 고양이 40%")과 실제 정답("고양이 100%")을 비교합니다. 이 둘의 차이를 **\'오차(Loss)\'** 또는 **\'손실\'**이라고 부릅니다. 정답과 멀수록 오차 점수는 커집니다.\n\n#### 3단계: 오답 노트 작성 및 공부법 찾기 (개선, Optimization)\n\n수험생은 틀린 문제를 그냥 넘어가지 않습니다. 왜 틀렸는지 분석하고, 다음에는 맞힐 수 있도록 공부 방향을 바꿉니다.\n\n*   **수험생:** "아, 뾰족한 귀와 긴 수염이 있으면

In [16]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론이죠! 인공지능 모델의 학습 원리를 **'처음으로 동물을 배우는 아이'**에 비유해서 아주 쉽게 설명해 드릴게요.

---

### **인공지능(AI) = 똑똑해지고 싶은 어린 아이**

여기 한 아이가 있습니다. 이 아이는 아직 '고양이'와 '강아지'를 구분하지 못해요. 우리는 이 아이가 동물을 잘 구분하도록 가르쳐야 합니다. 이 아이가 바로 **'인공지능 모델'**입니다.

---

### **학습의 4단계**

#### **1단계: 교재 준비하기 (데이터)**

아이를 가르치려면 먼저 '교재'가 필요하겠죠?
수많은 고양이 사진과 강아지 사진이 담긴 사진첩을 준비합니다. 중요한 것은 각 사진마다 **"이건 고양이야", "이건 강아지야"** 라고 정답(레이블)을 알려주는 것입니다.

*   **아이** = 인공지능 모델
*   **동물 사진첩** = 학습 데이터 (Training Data)
*   **"이건 고양이야" 라는 정답** = 레이블 (Label)

#### **2단계: 일단 찍어보기 (예측)**

이제 아이에게 고양이 사진 한 장을 보여줍니다. 아이는 아직 잘 모르기 때문에 그냥 찍어봅니다.

> **아이:** "음... 뾰족한 귀가 있으니까... **강아지!**"

이것이 인공지능의 **'첫 번째 예측(Prediction)'**입니다. 처음에는 거의 엉터리입니다.

#### **3단계: 오답노트 작성하기 (오차 계산)**

부모님(개발자)은 아이에게 정답을 알려줍니다.

> **부모님:** "땡! 이건 **고양이**란다. 네가 '강아지'라고 생각했으니 틀렸네."

아이는 자신이 얼마나 틀렸는지 깨닫습니다. '고양이'를 '강아지'라고 했으니, 꽤 많이 틀린 셈이죠.
인공지능의 세계에서는 이 '틀린 정도'를 **'오차(Error)' 또는 '손실(Loss)'**이라고 부릅니다. 오차가 클수록 많이 틀렸다는 뜻입니다.

#### **4단계: 스스로 고쳐나가기 (최적화)**

이제 가장 중요한 단계입니다. 아이는 왜 틀렸는지 스스로 생각해 봅니다.



### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [18]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=SecretStr(str(api_key)),
    temperature=0.5,
)

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [19]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [20]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

안녕하세요! 10년차 영어 선생님입니다. 식당에서 음식을 주문하는 상황은 정말 자주 겪게 되는 상황이죠. 자연스럽고 공손하게 주문할 수 있도록 대화를 구성해 봤어요.

---

**#상황: 식당에 가서 음식을 주문하고 싶어요**

**[FORMAT]**

- 영어 회화:
  **You (손님):** Excuse me, we're ready to order whenever you are.
  **Waiter/Waitress (직원):** Wonderful! What can I get for you today?
  **You (손님):** I'd like the Grilled Salmon, please. And could I get that with steamed vegetables instead of mashed potatoes?
  **Waiter/Waitress (직원):** Certainly. So that's one Grilled Salmon with steamed vegetables, and an iced tea. Anything else for you?
  **You (손님):** Yes, I'll also have a glass of iced tea.
  **You (손님):** No, that will be all for now, thank you.
  **Waiter/Waitress (직원):** Alright, your order will be out shortly!

- 한글 해석:
  **You (손님):** 실례합니다, 저희는 주문할 준비가 됐어요.
  **Waiter/Waitress (직원):** 좋아요! 오늘 무엇을 도와드릴까요? (무엇을 주문하시겠어요?)
  **You (손님):** 저는 구운 연어를 부탁드려요. 그리고 으깬 감자 대신 찐 야채로 주실 수 있을까요?
  **Waiter/Waitress (직원):** 물론이죠. 그럼 구운 연어 하나에 찐 야채, 그리고 아이스티 하나요. 더 필요한 건 없으신가요?
  **You (손님):

In [21]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

안녕하세요! 10년차 영어 선생님입니다. 식당에서 음식을 주문할 때 사용할 수 있는 유용한 표현들을 담은 대화입니다.

---

**#상황: 식당에서 음식을 주문하기**

**등장인물:**
*   **손님 (Customer)**: 당신
*   **종업원 (Server)**: 식당 종업원

---

- 영어 회화: **Server:** Good evening! Welcome to [Restaurant Name]. How many in your party?
- 한글 해석: **종업원:** 안녕하세요! [식당 이름]에 오신 것을 환영합니다. 몇 분이세요?

- 영어 회화: **Customer:** Good evening! Just one, please.
- 한글 해석: **손님:** 안녕하세요! 한 명입니다.

- 영어 회화: **Server:** Right this way. Here’s your menu. Can I get you something to drink while you look it over?
- 한글 해석: **종업원:** 이쪽으로 오세요. 메뉴 여기 있습니다. 메뉴를 보시는 동안 음료 먼저 드릴까요?

- 영어 회화: **Customer:** Yes, please. I'll have a glass of water, please.
- 한글 해석: **손님:** 네, 부탁드립니다. 물 한 잔 주세요.

- 영어 회화: **Server:** Certainly. Are you ready to order, or do you need a few more minutes?
- 한글 해석: **종업원:** 알겠습니다. 주문하시겠어요, 아니면 시간이 좀 더 필요하세요?

- 영어 회화: **Customer:** I'm ready to order, thank you. What do you recommend today?
- 한글 해석: **손님:** 주문할게요, 감사합니다. 오늘 어떤 걸 추천하시나요?

- 영어 회화: **Server:** Our special tod

In [22]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

네, 10년차 영어 선생님의 노하우를 담아 미국에서 피자 주문하는 상황의 영어 회화를 작성해 드릴게요.

---

# 상황: 미국에서 피자 주문

## 영어 회화:

**Pizza Palace Staff (P):** Thank you for calling Pizza Palace, how can I help you?
**Customer (C):** Hi, I'd like to place an order for delivery, please.

**P:** Okay, what can I get for you today?
**C:** I'd like a large pepperoni pizza.

**P:** And what kind of crust would you like? We have regular, thin, and hand-tossed.
**C:** Hmm, I'll go with thin crust, please.

**P:** Anything else with that? Maybe some wings or a drink?
**C:** Do you have any specials going on right now?

**P:** Yes, we have our "Family Feast" deal. It's a large 2-topping pizza, a side of breadsticks, and a 2-liter soda for $24.99.
**C:** Oh, that sounds good! Can I get that instead? For the pizza, I'll take half pepperoni, half supreme.

**P:** Sure. So that's a large, half pepperoni, half supreme pizza, with breadsticks and a 2-liter soda. Which soda would you like?
**C:** Coke, please.

**P:** Alright, and what's t